In [21]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import IDF

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("Tf-Idf")\
        .getOrCreate()

In [9]:
documentDF = spark.createDataFrame([
        (0,"Let's see an example of countVectorizer"),
        (1,"We will use pyspark library"),
        (2,"countVectorizer is important for NLP")
    ], ["index","sentence"])

In [13]:
documentDF.show(truncate=False)

+-----+---------------------------------------+
|index|sentence                               |
+-----+---------------------------------------+
|0    |Let's see an example of countVectorizer|
|1    |We will use pyspark library            |
|2    |countVectorizer is important for NLP   |
+-----+---------------------------------------+



In [17]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")


In [18]:
wordsDf = tokenizer.transform(documentDF)

In [20]:
wordsDf.show(truncate=False)

+-----+---------------------------------------+----------------------------------------------+
|index|sentence                               |words                                         |
+-----+---------------------------------------+----------------------------------------------+
|0    |Let's see an example of countVectorizer|[let's, see, an, example, of, countvectorizer]|
|1    |We will use pyspark library            |[we, will, use, pyspark, library]             |
|2    |countVectorizer is important for NLP   |[countvectorizer, is, important, for, nlp]    |
+-----+---------------------------------------+----------------------------------------------+



In [23]:
TF = HashingTF(inputCol="words", outputCol="term_frequency")

In [24]:

tf_data = TF.transform(wordsDf)

In [26]:
tf_data.show(truncate=False)

+-----+---------------------------------------+----------------------------------------------+---------------------------------------------------------------------------+
|index|sentence                               |words                                         |term_frequency                                                             |
+-----+---------------------------------------+----------------------------------------------+---------------------------------------------------------------------------+
|0    |Let's see an example of countVectorizer|[let's, see, an, example, of, countvectorizer]|(262144,[9639,86646,118651,194536,215995,222178],[1.0,1.0,1.0,1.0,1.0,1.0])|
|1    |We will use pyspark library            |[we, will, use, pyspark, library]             |(262144,[10645,89356,116873,124278,147489],[1.0,1.0,1.0,1.0,1.0])          |
|2    |countVectorizer is important for NLP   |[countvectorizer, is, important, for, nlp]    |(262144,[15889,16332,86646,116034,193401],[1.0,1.0,

In [27]:
idf = IDF(inputCol="term_frequency", outputCol="idf")

In [28]:
idf_Model = idf.fit(tf_data)

In [29]:
new_Data = idf_Model.transform(tf_data)

In [33]:
new_Data.select("index", "idf").show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|index|idf                                                                                                                                                                   |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |(262144,[9639,86646,118651,194536,215995,222178],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1    |(262144,[10645,89356,116873,124278,147489],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                          |
|2    |(262144,[15889,16332,86646,116034,193401],[0.6931471805599453,0.6931471805599453,0.28768207245178085,0.693147180559945

In [34]:
spark.stop()